In [28]:
import numpy as np
import nltk

n_paths = 5
alpha = 1
beta = 0.1

#phons = "abcdefghijklmnopqrstuvwxyz -"



phoneme_dict =   {#' ': 0,
                     u'AA': 0,
                     u'AE': 1,
                     u'AH': 2,
                     u'AO': 3,
                     u'AW': 4,
                     u'AY': 5,
                     u'B': 6,
                     u'CH': 7,
                     u'D': 8,
                     u'DH': 9,
                     u'EH': 10,
                     u'ER': 11,
                     u'EY': 12,
                     u'F': 13,
                     u'G': 14,
                     u'HH': 15,
                     u'IH': 16,
                     u'IY': 17,
                     u'JH': 18,
                     u'K': 19,
                     u'L': 20,
                     u'M': 21,
                     u'N': 22,
                     u'NG': 23,
                     u'OW': 24,
                     u'OY': 25,
                     u'P': 26,
                     u'R': 27,
                     u'S': 28,
                     u'SH': 29,
                     u'T': 30,
                     u'TH': 31,
                     u'UH': 32,
                     u'UW': 33,
                     u'V': 34,
                     u'W': 35,
                     u'Y': 36,
                     u'Z': 37,
                     u'ZH': 38,
                        "-": 39}
#removes stresses from vowels
def strip_stresses(phoneme):
    main_phon = phoneme[:-1]
    last_letter = phoneme[-1]
    if last_letter.isnumeric():
        return main_phon
    else:
        return phoneme
def obtain_phonemes(sentence, dictionary):
    sentence_phonemes = []
    for word in sentence:
        word = word.lower()
        if word in dictionary:
            #nltk.corpus.cmudict.dict() gives a list of lists, giving both
            #the american and english pronounciations
            phonemes = dictionary[word][0]
            stripped_phonemes = map(strip_stresses, phonemes)
            sentence_phonemes.extend(stripped_phonemes)
        else:
            #print "{} is not in the dictionary, can't get phonemes!".format(word)
            return None
    return sentence_phonemes
phons = phoneme_dict.keys()
D = nltk.corpus.cmudict.dict()
R = {k: " ".join(obtain_phonemes([k], D)) for k in D.keys()}

def stimes(factors):
    return np.exp(np.sum(np.log(factors)))
   
def sexp(base, exp):
    return np.exp(np.log(base) * exp)


def sdiv(a,b):
    return np.exp(np.log(a)-np.log(b))

def phonemes_to_homophones(p):
    return R.get(p)

# Compute the transition probability
def get_p_w_p_star(w, p):
    # It is critical that the regex has \s at the end.
    # This prevents p from matching with words for which it is a non-strict prefix
    wprimes = [word for word in words if re.match(r"^"+p+"\s", " ".join(obtain_phonemes(word)))]
    p_w_wprimes = [lm(wprime, w) for wprime in wprimes]
    return np.sum(p_w_wprimes)

def last(w,p):
    list_w = w.split()
    list_p = p.split()
    if list_p:
        return list_p[-1]
    else:
        return list_w[-1]
def hat(w,p):
    if p:
        return (w, " ".join(p.split(" ")[:-1]))
    elif w:
        return (" ".join(obtain_phonemes(w.split(" "))[:-1]),p)
    else:
        return w,p
    
def top_keys(d, k, beta):
    sentences = [a[0] for a in d.keys()]
    logprobs = np.asarray(d.values())
    string_lens = np.asarray([len(sentence)+1 for sentence in sentences], dtype=float)
    
    top_k_idxs = argsort(logprobs / string_len)[-k:] # dividing makes them greater because they're negative!
    top_k_keys = [d.keys()[i] for i in top_k_idxs]
    return top_k_keys


In [35]:

def beam_search(phon_probs):
    Z = [[] for t in range(T)]
    p_b = [{} for t in range(T)]
    p_nb = [{} for t in range(T)]
    p = [{} for t in range(T)]
    p_w_p = [{} for t in range(T)]
    # Just an initialization that works...
    p_nb[0] = {("",""):1.}
    p_b[0] = {("",""): 0.}
    p[0] = {("",""):1.}

    Z[0] = [("", "")]
    T = phon_probs.shape[0]

    # Iterate over input timesteps
    for t in range(1, T+1):
        print "t={}".format(t)
        # Consider all of the (w,p) pairs which were candidates given data up to time t-1
        # and compute the probabilities of 'some' new strings given data up to time t
        # 'some' = the current strings and 1-phoneme extensions of them
        for (w,p) in Z[t-1]:
            print "considering ({},{})"
            # We need to compute:
            
            # 1. p_nb[t][(w,p)]
            # 2. p_b[t][(w,p)]
            # 3. p_nb[t][(w,p + k)] for all k not blank. Remembering that this may lead to a new word.
           
            # Consider the extension by a blank phoneme
            # This always leaves (w,p) as it was
            # So we are assigning p_nb[t][(w,p)] and p_b[t][(w,p)]
            p_nb_t_add = 
            if p_nb[t].get((w,p)):
                
            p_nb[t][(w,p)] = stimes([p_nb[t-1][(w,p)], phon_probs[t, phoneme_dict[last(w,p)]]]) if (w,p)!=("","") else 0. 
            if hat(w,p) in Z[t-1]:
                
            print "p_b[{}] =".format(t), p_b[t]
            print "p[{}] =".format(t-1), p[t-1]
            p_b[t][(w,p)] =  stimes([p[t-1][(w,p)], phon_probs[t, phoneme_dict["-"]]])
            
                
            # Consider extensions of the uncontracted thing by one phoneme
            for k in phons[:-1]:
                
                # When the extension definitely adds a phoneme to the contraction
                # all the probs are nb because we aren't adding blank!
                if k!="-" and k != last(w,p):

                    # if p+k can be read as a word v, compute the probability of (wv,0) using the lm
                    # for every such v
                    # Note that (-p_w_p_star[(w,p)])*alpha exactly cancels the same term in a prev computation of p[t-1][(w,p)]
                    # This is because (unlogarithmed) p[t-1] can be factored into acoustic and language factors
                    # where exp(p_w_p_star[(w,p)]*alpha) is the language factor
                    # The new language model factor is just the lm evaluated on v and the sentence
                    for h in R.get(p+" "+k):
                        p_nb[t][(w+" "+h, "")] = stimes([phon_probs[t, phoneme_dict[k]],
                                                  p[t-1][(w,p)],
                                                  sexp(sdiv(lm(h, w),p_w_p_star[(w,p)]), alpha)])
                        
                        p_b[t][(w+" "+h, "")] =  0.
                              
                    # Compute the probability of (w,p+k) under the language model, NOT CONSIDERING THE ACOUSTIC MODEL
                    # This is the sum of probabilities of wx where p is a strict prefix of every x
                    p_w_p_star[(w, p+k)] = get_p_w_p_star(w, p+k)
                            
                    p_nb[t][(w,p+k)] = stimes([phon_probs[t, phoneme_dict[k]],
                                        p[t-1][(w,p)],
                                        sexp(sdiv(p_w_p_star[(w,p+k)], p_w_p_star[(w,p)]), alpha)])
                            
                # When extension is the same as the final phoneme
                # the extension only extends y if the final phoneme in the uncontracted thing is blank
                elif k!="-" and k==last(w,p):
                    p_nb[t][(w,p+k)] = stimes([phon_probs[t, phoneme_dict[k]],
                                       p_b[t-1][(w,p)],
                                        sexp(sdiv(p_w_p_star[(w,p+k)], p_w_p_star[(w,p)]), alpha)])
                else: # k == "-"
                    continue
                
            # Set the probability of a string given input data up to time t as the sum of the marginals over t'th model out
            p[t] = {y: p_b[t][y] + p_nb[t][y] for y in p_b[t].keys()}
            # Take the top k strings 
            Z[t] = top_keys(p[t], n_paths, beta)
    return Z[T]
    # Don't forget to throw away the p's when you're done!
    

In [36]:
# phon_probs = 
beam_search(np.load("phoneme_probs.npy"))

t=1
p_b[1] = {}
p[0] =

IndexError: string index out of range